In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv("outputs/cleaned_matched_df.csv")
df

,ID,Topic,ReversedDomain,Content
0,362,Business,ac.accent,"['accent', 'services', 'uk', 'based', 'full', ..."
1,362,Regional,ac.accent,"['accent', 'services', 'a', 'full', 'service',..."
2,383,Regional,ac.acs,"['anderson', 'county', 'schools', 'public', 's..."
3,390,Games,ac.adamcadre,"['allows', 'the', 'game', 'to', 'be', 'played'..."
4,424,World,ac.aikido,"['langnau', 'aikidogruppe', 'in', 'zusammenarb..."
...,...,...,...,...
2289392,91033952,Society,zw.org.nascoh,"['national', 'association', 'of', 'societies',..."
2289393,91034085,Computers,zw.org.zispa,"['zw', 'domain', 'zimbabwe', 'nic', 'for', 'zw..."
2289394,91034085,Regional,zw.org.zispa,"['zimbabwe', 'internet', 'service', 'providers..."
2289395,91034088,Regional,zw.org.zlhr,"['zimbabwe', 'lawyers', 'for', 'human', 'right..."


In [40]:
valid_nodes = pd.DataFrame({"node_id": df["ID"]}).drop_duplicates().reset_index(drop=True)

In [50]:
valid_nodes

,node_id
0,362
1,383
2,390
3,424
4,442
...,...
2089095,91033860
2089096,91033932
2089097,91033952
2089098,91034085


## Save edges to database

In [22]:
edgesdomainpath = "webdata/commoncrawl/cc-main-2017-may-jun-jul/edges.txt"

In [5]:
conn = sqlite3.connect("outputs/edges.db")
cursor = conn.cursor()

In [6]:
# Create table for edges
cursor.execute("""
    CREATE TABLE edges (
        from_node INTEGER,
        to_node INTEGER
    )
""")
conn.commit()

In [7]:
chunk_size = 10**6  # Adjust based on memory
with pd.read_csv(edgesdomainpath, sep="\t", chunksize=chunk_size, header=None, names=["from_node", "to_node"]) as reader:
    for chunk in reader:
        chunk.to_sql("edges", conn, if_exists="append", index=False)

In [39]:
valid_nodes.to_sql("valid_nodes", conn, if_exists="replace", index=False)

In [9]:
# Create indexes on the edges and valid_nodes tables
cursor.execute("CREATE INDEX idx_edges_from_node ON edges (from_node);")
cursor.execute("CREATE INDEX idx_edges_to_node ON edges (to_node);")
conn.commit()

In [72]:
cursor.execute("CREATE INDEX idx_valid_nodes_node_id ON valid_nodes (node_id);")
conn.commit()

# Create filtered database

### Remove nodes out of range

In [33]:
conn = sqlite3.connect("outputs/edges.db")
cursor = conn.cursor()

In [3]:
# Step 1: Get the minimum and maximum valid node IDs
cursor.execute("SELECT MIN(node_id), MAX(node_id) FROM valid_nodes;")
min_valid_id, max_valid_id = cursor.fetchone()
print(f"Valid node ID range: {min_valid_id} - {max_valid_id}")

cursor.execute("""
DELETE FROM edges
WHERE from_node < ? OR from_node > ?;
""", (min_valid_id, max_valid_id))

cursor.execute("""
DELETE FROM edges
WHERE to_node < ? OR to_node > ?;
""", (min_valid_id, max_valid_id))

conn.commit()

Valid node ID range: 362 - 91034088


In [8]:
# Get the total number of rows in the edges table
cursor.execute("SELECT COUNT(*) FROM edges;")
total_rows = cursor.fetchone()[0]
print(total_rows)

#1071170354

1071170354


### Filter based on the from_nodes (do in chunks due to size)

In [9]:
chunk_size = 5*10**8
offset = 0
query = f"""
    CREATE TABLE temp_edges AS
    SELECT *
    FROM edges
    LIMIT {chunk_size} OFFSET {offset};
"""
cursor.execute(query)
conn.commit()
print(f"Processed rows: {min(offset + chunk_size, total_rows)}")

Processed rows: 500000000


In [10]:
query = f"""
    CREATE TABLE filtered_from_p1 AS
    SELECT temp_edges.*
    FROM temp_edges
    INNER JOIN valid_nodes 
    ON temp_edges.from_node = valid_nodes.node_id;
"""

cursor.execute(query)
conn.commit()
print(f"Processed rows: {min(offset + chunk_size, total_rows)}")

Processed rows: 500000000


In [11]:
cursor.execute("SELECT COUNT(*) FROM filtered_from_p1;")
part1_from_rows = cursor.fetchone()[0]
print(part1_from_rows)

81509929


In [14]:
cursor.execute("SELECT * FROM filtered_from_p1 LIMIT 20 OFFSET 81509909;")
part1_from_rows = cursor.fetchall()
for row in part1_from_rows:
    print(row)

(42620222, 23612737)
(42620222, 24229864)
(42620222, 24230475)
(42620222, 42264752)
(42620222, 48714451)
(42620252, 11326342)
(42620252, 21860195)
(42620252, 24230475)
(42620252, 24231253)
(42620252, 34369078)
(42620252, 34601565)
(42620252, 36928002)
(42620252, 45612695)
(42620252, 45880772)
(42620252, 62632273)
(42620252, 87543940)
(42620252, 87625743)
(42620252, 88534673)
(42620252, 88636058)
(42620312, 24231253)


In [15]:
cursor.execute("DROP TABLE IF EXISTS temp_edges;")
conn.commit()

In [16]:
query = f"""
    CREATE TABLE temp_edges AS
    SELECT *
    FROM edges
    LIMIT {total_rows - chunk_size} OFFSET {chunk_size};
"""
cursor.execute(query)
conn.commit()
print(f"Processed rows: {min(offset + chunk_size, total_rows)}")

Processed rows: 500000000


In [19]:
query = f"""
    CREATE TABLE filtered_from_p2 AS
    SELECT temp_edges.*
    FROM temp_edges
    INNER JOIN valid_nodes 
    ON temp_edges.from_node = valid_nodes.node_id;
"""

cursor.execute(query)
conn.commit()
print(f"Processed rows: {min(total_rows - chunk_size, total_rows)}")

Processed rows: 571170354


In [20]:
cursor.execute("SELECT COUNT(*) FROM filtered_from_p2;")
part2_from_rows = cursor.fetchone()[0]
print(part2_from_rows)

120540544


In [21]:
cursor.execute("SELECT * FROM filtered_from_p2 LIMIT 10;")
part2_from_rows = cursor.fetchall()
for row in part2_from_rows:
    print(row)

(42620443, 21860195)
(42620443, 24621514)
(42620443, 34391867)
(42620503, 9266025)
(42620503, 9795978)
(42620503, 11674717)
(42620503, 12851052)
(42620503, 16141626)
(42620503, 16233566)
(42620503, 18162839)


In [22]:
query = """
    CREATE TABLE merged_filtered_from AS
    SELECT * FROM filtered_from_p1
    UNION ALL
    SELECT * FROM filtered_from_p2;
"""
cursor.execute(query)
conn.commit()

In [23]:
cursor.execute("SELECT COUNT(*) FROM merged_filtered_from;")
merged_from_rows = cursor.fetchone()[0]
print(merged_from_rows)

202050473


### Now filter based on the to_nodes (can do all at once due to reduced size)

In [30]:
query = f"""
    CREATE TABLE filtered_edges AS
    SELECT merged_filtered_from.*
    FROM merged_filtered_from
    INNER JOIN valid_nodes 
    ON merged_filtered_from.to_node = valid_nodes.node_id;
"""

cursor.execute(query)
conn.commit()

In [31]:
cursor.execute("SELECT COUNT(*) FROM filtered_edges;")
filtered_rows = cursor.fetchone()[0]
print(filtered_rows)

68294063


# Save filtered_edges table

In [34]:
filtered_edges_df = pd.read_sql_query("SELECT * FROM filtered_edges", conn)

In [36]:
filtered_edges_df

,from_node,to_node
0,362,13833969
1,362,21183718
2,362,24229864
3,362,30617333
4,362,34548414
...,...,...
68294058,91034088,45612695
68294059,91034088,48714451
68294060,91034088,66662916
68294061,91034088,67027618


In [35]:
filtered_edges_df.to_csv("outputs/filtered_edges_df.csv", index=False)